In [3]:
# Pandas is used for data manipulation
import pandas as pd
import numpy as np

# Read in data
z_data = pd.read_csv('fse17_act_deact_zscore_groups.csv',sep=';',decimal='.')

# Remove irrelevant columns
z_data.drop(list(z_data.filter(regex = '_act')),axis = 1, inplace = True)
z_data.drop(list(z_data.filter(regex = 'aggr')),axis = 1, inplace = True)
data = z_data# necessary, because task and proband are dropped for the learning

#we need the list with the labels to test the predictions; everytime the label or proband changes, we add it to the list
labelsComputingAccuracy = []

condition = z_data.at[0, 'task']
participant = z_data.at[0, 'proband']
    
for i in range(len(z_data)):
    if condition != z_data.at[i, 'task'] or participant != z_data.at[i, 'proband']:
#         print(participant,i)
        labelsComputingAccuracy.append(condition)
        condition = z_data.at[i, 'task']
        participant = z_data.at[i, 'proband']
#         print(participant,i)
labelsComputingAccuracy.append(condition)

# recode word labels to num labels so t_pot can handle the lables =======
labels = []
for i in range(len(z_data)):
    if z_data.at[i,'task'] == 0:
        labels.append(0)
    if z_data.at[i,'task'] == 1:
        labels.append(1)
    if z_data.at[i,'task'] == 2:
        labels.append(2)

#=== remove everything that is not a feature ===================================================
z_data = z_data.drop('proband', axis = 1)
z_data = z_data.drop('scan', axis = 1)
z_data = z_data.drop('trial', axis = 1)
z_data = z_data.drop('task', axis = 1)
z_data = z_data.drop('snippet', axis = 1)
z_data = z_data.drop('response', axis = 1)

#=== creating training and validation set ======================================================
# Saving feature names for later use
feature_list = z_data.columns

# Convert to numpy array
features = np.array(z_data)
labels = np.array(labels)

#split the data into train and validation set
#get the rows excluding the last indexed (i.e., row with the index 10973 is the last to include)
#the rows until 10973 contain the first 12 participants, the remaining rows the last 4
training_features = features[0:9570]
training_features = np.array(training_features)
testing_features = features[9570:12180]
testing_features = np.array(testing_features)

training_target = labels[0:9570]
training_target = np.array(training_target)
testing_target = labels[9570:12180]
testing_target = np.array(testing_target)

KeyboardInterrupt: 

In [2]:
from tpot import TPOTClassifier
fileNameTPot = 'Deact/tpot_mnist_pipeline_NotAggregatedParticipantSplit'

# tpot = TPOTClassifier(generations=5, population_size=20, verbosity=2, n_jobs=20)
# tpot.fit(training_features, training_target)
# print(tpot.score(testing_features, testing_target))
# tpot.export(fileNameTPot + '.py')

C:\Users\author\Anaconda3\lib\site-packages\deap\tools\_hypervolume\pyhv.py:33: ImportWarning: Falling back to the python version of hypervolume module. Expect this to be very slow.
  "module. Expect this to be very slow.", ImportWarning)
C:\Users\author\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Users\author\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Users\author\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Users\author\Anaconda3\lib\importlib\_bootstrap_external.py:426: ImportWarning: Not importing directory C:\Users\author\Anaconda3\lib\site-packages\mpl_toolkits: missing __init__
  _warnin

In [16]:
with open(fileNameTPot + '.py') as f:
    content = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
# content = [x.strip() for x in content] 

#      or 'exported_pipeline = ' not in line
cleanedContent = []
for line in content:
    if 'tpot_data' not in line and 'training_target, testing_target' not in line:
        cleanedContent.append(line)
#         content.remove(line)

fileForLearning = fileNameTPot + '_cleaned.py'
with open(fileForLearning, 'w') as filehandle:  
    for line in cleanedContent:
        filehandle.write('%s\n' % line)

In [18]:
%run -i $fileForLearning

In [20]:
z_data = data # necessary, because task and proband are dropped for the learning
condition = z_data.at[9570, 'task']
participant = z_data.at[9570, 'proband']
majorityVote = []
labelsForAcc = []
countComprehension = 0
countRest = 0
    
for i,j in zip(range(9570,len(z_data)),range(len(results))):
    if condition != z_data.at[i, 'task'] or participant != z_data.at[i, 'proband']:
        labelsForAcc.append(condition)
        condition = z_data.at[i, 'task']
        participant = z_data.at[i, 'proband']
        if countComprehension > countRest:
            majorityVote.append('C')
        elif countRest > countComprehension:
            majorityVote.append('R')
        else:
            majorityVote.append('CR')
        countComprehension = 0
        countRest = 0

    if (results[j] == 0):
        countComprehension = countComprehension + 1
    if (results[j] == 1):
        countRest = countRest + 1

labelsForAcc.append(condition)
if (results[j] == 0):
    countComprehension = countComprehension + 1
if (results[j] == 1):
    countRest = countRest + 1

if countComprehension > countRest:
    majorityVote.append('C')
elif countRest > countComprehension:
    majorityVote.append('R')
else:
    majorityVote.append('CR')


In [22]:
labelsForAcc = pd.Series(labelsForAcc)
labelsForAcc = labelsForAcc.replace({0: 'C', 1: 'R', 2: 'S'})

num_matches = 0;
for a, b in zip(majorityVote, labelsForAcc):
    if b in a:
        num_matches = num_matches + 1
print('Number of matches:',num_matches,'(of',len(labelsForAcc),')')

accuracy = num_matches/len(labelsForAcc)*100
print('Accuary: ',accuracy)


Number of matches: 112 (of 165 )
Accuary:  67.87878787878789


In [23]:
with open('listOfAccuracies.txt','a+') as f:
    f.write('%s\n' % 'NoAgg_Deact' + str(accuracy) + '\n')
